In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#importing seaborn for statistical plots
import seaborn as sns
# Import Linear Regression machine learning library
from sklearn.linear_model import LinearRegression

import statsmodels.formula.api as smf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


1. Read and explore the given dataset.  ( Rename column/add headers, plot histograms, find data characteristics)

In [5]:
!pip install scikit-surprise

     |████████████████████████████████| 3.3MB 4.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.0.6-cp36-cp36m-linux_x86_64.whl size=1683516 sha256=d754aeff4846e783202b1c7ca67064f7de3981a0f5f6b5f329adbd349cb724ae
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
#recomm_df = pd.read_csv('https://drive.google.com/file/d/1ClBptsK3V5KgKXtK2GSRzFNAW7GnTPDW/view?usp=sharing')
recomm_df = pd.read_csv('/content/drive/My Drive/AIML/5thResidency/Recommandation/Recommandation1/Lab/ratings_Electronics.csv',names=["userId","productId", "ratings" , "timestamp"])


In [7]:
recomm_df.shape
recomm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048576 entries, 0 to 1048575
Data columns (total 4 columns):
userId       1048576 non-null object
productId    1048576 non-null object
ratings      1048576 non-null int64
timestamp    1048576 non-null int64
dtypes: int64(2), object(2)
memory usage: 32.0+ MB


In [0]:
#drop the columns id, date, zipcode
recomm_df = recomm_df.drop(["timestamp"],axis=1)

In [9]:
recomm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048576 entries, 0 to 1048575
Data columns (total 3 columns):
userId       1048576 non-null object
productId    1048576 non-null object
ratings      1048576 non-null int64
dtypes: int64(1), object(2)
memory usage: 24.0+ MB


2. Take a subset of the dataset to make it less sparse/ denser. ( For example, keep the users only who has given 50 or more number of ratings )

In [0]:
# get the number of review by userId
users_by_count = recomm_df.groupby('userId')['productId'].count().reset_index(name='counts') 

In [0]:
#consider only users having  more than 50 count
top_users = users_by_count[ users_by_count['counts'] > 50]


In [12]:
from surprise.model_selection import train_test_split
#surprise.model_selection.split.train_test_split
top_users_reviews = recomm_df[recomm_df['userId'].isin(top_users['userId'].values)]
top_users_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3406 entries, 2162 to 1048158
Data columns (total 3 columns):
userId       3406 non-null object
productId    3406 non-null object
ratings      3406 non-null int64
dtypes: int64(1), object(2)
memory usage: 106.4+ KB


In [13]:
%matplotlib inline

import pandas
from sklearn.model_selection import train_test_split
import numpy as np
import time
from sklearn.externals import joblib


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [14]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(top_users_reviews[['userId', 'productId', 'ratings']], reader)
data.read_ratings

<bound method Dataset.read_ratings of <surprise.dataset.DatasetAutoFolds object at 0x7f7f7ba5ceb8>>

3. Split the data randomly into train and test dataset. ( For example, split it in 70/30 ratio)

In [0]:
# spliting the data into  train_data, test_data
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.30)

4. Build Popularity Recommender model.

In [16]:
# Building most popular recommedation model
wavg = top_users_reviews.groupby('productId')['ratings'].mean().reset_index(name='avg rating').sort_values('avg rating', ascending=False)
wavg.head(10)

,productId,avg rating
1453,B0000AE6C3,5.0
1234,B00008XETK,5.0
1248,B00009017A,5.0
1247,B00008ZPNS,5.0
1245,B00008ZPI1,5.0
2288,B0007WK8LC,5.0
1243,B00008ZLHG,5.0
2289,B0007WZMW2,5.0
2295,B0007XOHTU,5.0
2296,B0007XP8GQ,5.0


5. Build Collaborative Filtering model.

In [17]:
 from surprise import SVD, accuracy
#from surprise.cross_validation import cross_validate
algo = SVD()
algo.fit(trainset)
#cross_validate(algo, tain_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [0]:
predictions = algo.test(testset)

In [19]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 1.0727


1.072660578539504

6. Evaluate both the models. ( Once the model is trained on the training data, it can be used to compute the error (like RMSE) on predictions made on the test data.) You can also use a different method to evaluate the models.

In [20]:
import surprise
from surprise import evaluate

surprise.evaluate(algo, data, measures=[u'rmse', u'mae'], with_dump=False)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/lib/python3.6/dist-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


RMSE: 1.0538
MAE:  0.7835
------------
Fold 2
RMSE: 1.0400
MAE:  0.7877
------------
Fold 3
RMSE: 1.0442
MAE:  0.7859
------------
Fold 4
RMSE: 1.1080
MAE:  0.8328
------------
Fold 5
RMSE: 1.0180
MAE:  0.7772
------------
------------
Mean RMSE: 1.0528
Mean MAE : 0.7934
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.7835478045111721,
                             0.7876959716674671,
                             0.7858804133969505,
                             0.8327644436729992,
                             0.7772127086024673],
                            'rmse': [1.0538083076194333,
                             1.0400327578816937,
                             1.0441537028750474,
                             1.10804910326716,
                             1.0180012537666148]})

7. Get top - K ( K = 5) recommendations. Since our goal is to recommend new products to each user based on his/her habits, we will recommend 5 new products.


In [0]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]



In [22]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
857,A25HBO5V8S8SEA,B0006OJG78,5.0,5.000000,{'was_impossible': False},120,1,0.000000
383,A149RNR5RH19YY,B00008I9K8,4.0,3.991089,{'was_impossible': False},49,2,0.008911
255,AKT8TGIT6VVZ5,B000021YU8,5.0,4.990988,{'was_impossible': False},96,1,0.009012
247,A5JLAU2ARJ0BO,B000EEQ83K,4.0,4.010649,{'was_impossible': False},295,1,0.010649
141,A2BGZ52M908MJY,B00005ALB1,4.0,4.019001,{'was_impossible': False},35,1,0.019001
174,A25HBO5V8S8SEA,B00006JI6U,5.0,4.959176,{'was_impossible': False},120,2,0.040824
1021,ABH9D1TEOJ56E,B00006JLOS,4.0,4.046085,{'was_impossible': False},39,0,0.046085
197,ABH9D1TEOJ56E,B000087BX8,4.0,4.046085,{'was_impossible': False},39,0,0.046085
954,ABH9D1TEOJ56E,B0000CBJ8J,4.0,4.046085,{'was_impossible': False},39,0,0.046085
594,ABH9D1TEOJ56E,B00008O35W,4.0,4.046085,{'was_impossible': False},39,0,0.046085


8. Summarise your insights.

In [23]:
print("Since the given data is huge. I have clence the  data to take the advantage of the recommendation system.")
print("And used Surprise library for getting best pediction based on highest number of users who has rated high(5 rating).")
print("This has achieved by collabrative filtering, since it supports different prediction algo.")

Since the given data is huge. I have clence the  data to take the advantage of the recommendation system.
And used Surprise library for getting best pediction based on highest number of users who has rated high(5 rating).
This has achieved by collabrative filtering, since it supports different prediction algo.
